## REST API

REST stands for representational state transfer and is a software architecture style that defines a pattern for client and server communications over a network.

REST APIs provide access to web service data through public web URLs. 
You access data from a REST API by sending an HTTP request to a specific URL and processing the response.

REST APIs listen for HTTP methods like GET, POST, and DELETE to know which operations to perform on the web service’s resources. A resource is any data available in the web service that can be accessed and manipulated with HTTP requests to the REST API. The HTTP method tells the API which action to perform on the resource.

| HTTP Methods  |Description                              |
|:--------------|:----------------------------------------|
| GET           | Retrieve an existing resource.          |
| POST          | Create a new resource.                  |
| PUT           | Update an existing resource.            |
| PATCH         | Partially update an existing resource.  |
| DELETE        | Delete a resource.                      |


Once a REST API receives and processes an HTTP request, it will return an HTTP response. Included in this response is an HTTP status code. This code provides information about the results of the request. An application sending requests to the API can check the status code and perform actions based on the result. These actions could include handling errors or displaying a success message to a user.


|Status Codes |Meaning	               | Description                                                                    |
|:------------|:-----------------------|:-------------------------------------------------------------------------------|
| 200         | OK	                   | The requested action was successful.                                           |
| 201	      | Created	               | A new resource was created.                                                    |
| 202	      | Accepted	           | The request was received, but no modification has been made yet.               |
| 204	      | No Content	           | The request was successful, but the response has no content.                   |
| 400         | Bad Request	           | The request was malformed.                                                     |
| 401	      | Unauthorized	       | The client is not authorized to perform the requested action.                  |
| 404	      | Not Found	           | The requested resource was not found.                                          |
| 415	      | Unsupported Media Type | The request data format is not supported by the server.                        |
| 422         | Unprocessable Entity   | The request data was properly formatted but contained invalid or missing data. |
| 500	      | Internal Server Error  | The server threw an error when processing the request.                         |


REST APIs for weather documentation: <br>
https://www.weather.gov/documentation/services-web-api <br>
https://www.ncdc.noaa.gov/cdo-web/webservices/v2 and token: https://www.ncdc.noaa.gov/cdo-web/token

Links for additional weather.gov api information: https://training.weather.gov/nwstc/sysinfo/sites.html and https://forecast.weather.gov/stations.php

# NWS API

In [ ]:
import requests

In [ ]:
#Pleasant Hill forcasting office for KC region
response = requests.get("https://api.weather.gov/offices/EAX/")
print(response.json())
print(response.status_code)

In [ ]:
# MCI latest observations
response = requests.get("https://api.weather.gov/stations/KMCI/observations/latest") 
print(response.json())
print(response.status_code)

In [ ]:
# MCI observations between dates
response = requests.get("https://api.weather.gov/stations/KMCI/observations?start=2023-09-05T00:00:00.000Z&end=2023-09-10T00:00:00.000Z") # MCI observations between dates
print(response.json())
print(response.status_code)

In [ ]:
data = response.content
with open('data.json', 'wb') as f:
    f.write(data)

# NOAA API

In [ ]:
import requests

In [ ]:
# Data from MCI during date range
response = requests.get("https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:USC00215638&startdate=2020-01-01&enddate=2020-03-31&limit=1000", headers={'token': 'token'})
print(response.json())
print(response.status_code)

In [ ]:
# or
import requests

token = "token"
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
start = "2019-01-01"
end = "2019-01-31"
params = {
    "datasetid": "GHCND",
    "stationid": "GHCND:USC00215638",
    "units": "standard",
    "startdate": start,
    "enddate": end,
    "limit": 1_000,
}
response = requests.get(url, params=params, headers={"Token": token})
print(response.json())

In [ ]:
data = response.content
with open('dataNOAA.json', 'wb') as f:
    f.write(data)

# NOAA API Looping

In [15]:
import requests
import pandas as pd
start_date, end_date = '2015-01-01', '2020-12-31'
start_goal = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m-%d')
end_goal = pd.date_range(start=start_date, end=end_date, freq='M') .strftime('%Y-%m-%d')

token = "token"
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
data  = []
for i in range(0,start_goal.size):
    start = start_goal[i]
    end = end_goal[i]
    params = {
        "datasetid": "GHCND",
        "stationid": "GHCND:USC00215638",
        "units": "standard",
        "startdate": start,
        "enddate": end,
        "limit": 1_000}
    response = requests.get(url, params=params, headers={"Token": token}).json()
    data.append(response)
    working_data = pd.json_normalize(data, record_path='results')

In [16]:
working_data

,date,datatype,station,attributes,value
0,2015-01-01T00:00:00,PRCP,GHCND:USC00215638,",,7,0800",0.0
1,2015-01-01T00:00:00,SN31,GHCND:USC00215638,",,7,0800",16.0
2,2015-01-01T00:00:00,SN32,GHCND:USC00215638,",,7,0800",14.0
3,2015-01-01T00:00:00,SN33,GHCND:USC00215638,",,7,0800",25.0
4,2015-01-01T00:00:00,SNOW,GHCND:USC00215638,",,7,",0.0
...,...,...,...,...,...
23203,2020-12-31T00:00:00,SX31,GHCND:USC00215638,",,7,0800",31.0
23204,2020-12-31T00:00:00,SX33,GHCND:USC00215638,",,7,0800",31.0
23205,2020-12-31T00:00:00,TMAX,GHCND:USC00215638,",,7,0800",21.0
23206,2020-12-31T00:00:00,TMIN,GHCND:USC00215638,",,7,0800",6.0


In [17]:
df = working_data.drop(columns=['station', 'attributes'])
df = df.pivot(index='date', columns='datatype', values='value')
df

datatype,EVAP,PRCP,SN31,SN32,SN33,SNOW,SNWD,SX31,SX32,SX33,TMAX,TMIN,TOBS,WESD
date,,,,,,,,,,,,,,
2015-01-01T00:00:00,NaN,0.00,16.0,14.0,25.0,0.0,1.0,24.0,23.0,27.0,23.0,-1.0,23.0,NaN
2015-01-02T00:00:00,NaN,0.00,22.0,21.0,27.0,0.0,0.0,28.0,28.0,29.0,29.0,6.0,17.0,NaN
2015-01-03T00:00:00,NaN,0.00,23.0,23.0,27.0,0.0,0.0,29.0,29.0,29.0,32.0,17.0,21.0,NaN
2015-01-04T00:00:00,NaN,0.00,13.0,10.0,23.0,0.0,0.0,26.0,26.0,29.0,23.0,-12.0,-11.0,NaN
2015-01-05T00:00:00,NaN,0.00,9.0,7.0,20.0,0.0,0.0,16.0,15.0,23.0,0.0,-12.0,-12.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27T00:00:00,NaN,0.00,26.0,NaN,31.0,0.0,0.0,31.0,NaN,31.0,25.0,8.0,22.0,NaN
2020-12-28T00:00:00,NaN,0.03,27.0,NaN,31.0,1.0,1.0,30.0,NaN,31.0,24.0,7.0,18.0,NaN
2020-12-29T00:00:00,NaN,0.00,25.0,NaN,31.0,0.0,1.0,30.0,NaN,31.0,19.0,-8.0,6.0,NaN
